In [75]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Package

In [76]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import random

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data load

In [79]:
# DB 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [80]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

In [81]:
df = sql_to_df('MainData_식생')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   위도                497 non-null    float64
 2   경도                497 non-null    float64
 3   생태통로_유형           497 non-null    int64  
 4   등산객_이용빈도          497 non-null    int64  
 5   생태통로_효율성          497 non-null    int64  
 6   이용확인종_수           497 non-null    int64  
 7   폭(m)              309 non-null    float64
 8   연장(m)             309 non-null    float64
 9   높이(m)             44 non-null     float64
 10  개방도               44 non-null     float64
 11  경사도               497 non-null    object 
 12  주변 로드킬 빈도         497 non-null    int64  
 13  산책로까지의 최단 거리(km)  497 non-null    float64
 14  농가까지의 거리(km)      497 non-null    float64
 15  도로 최고제한속도(km/h)   497 non-null    int64  
 16  주변 동물종 개수         497 non-null    int64  
 1

## Translation columns name Korean to English

In [82]:
rename_col = {'번호' : 'num', 
              '위도' : 'latitude',
              '경도' : 'longitude',
              '생태통로_유형' : 'type',
              '등산객_이용빈도' : 'frequency_use_hikers',
              '생태통로_효율성' : 'efficiency',
              '이용확인종_수' : 'num_use_species',
              '폭(m)' : 'width',
              '연장(m)' : 'length',
              '높이(m)' : 'hiehgt',
              '개방도' : 'openness',
              '경사도' : 'solid_slope',
              '주변 로드킬 빈도' : 'frequency_roadkill',
              '산책로까지의 최단 거리(km)' : 'dist_walkway',
              '농가까지의 거리(km)' : 'dist_farm',
              '도로 최고제한속도(km/h)' : 'speed_limit',
              '주변 동물종 개수' : 'num_near_species',
              '주변동물 출현빈도' : 'frequency_appearance_animals',
              '지형기호(2.3km)' : 'topography',
              '하천거리(km)' : 'dist_river',
              '식생' : 'vegetation', 
              '등산로까지 최단거리(km)' : 'dist_hikingtrails',
              '유도울타리_위도_2.3km' : 'fence_latitude',
              '유도울타리_경도_2.3km' : 'fence_longitude',
              '유도울타리_연장_m' : 'fence_length',
              '유도울타리_높이_m' : 'fence_height',
              '교통량' : 'traffic_volumne',
              '환경영향평가점수' : 'environmental_score', ## 국토환경성평가. 환경영향평가x
              '건물까지거리(km)' : 'dist_buildings',
              '차선_수' : 'num_lane'}

df.rename(columns=rename_col, inplace=True)

## Labeling

topography : X(특성없음), 하천, 산지, 기타, 해안, 화산

In [83]:
df['topography'] = [x.split('-')[0] for x in df['topography']]
df['topography'].unique()

array(['X', '하천', '산지', '기타', '해안', '화산'], dtype=object)

efficiency : 0(낮음)/1(일정 수준 이상)

In [84]:
df['efficiency'] = [1 if x==2 else x for x in df['efficiency']]

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   num                           497 non-null    int64  
 1   latitude                      497 non-null    float64
 2   longitude                     497 non-null    float64
 3   type                          497 non-null    int64  
 4   frequency_use_hikers          497 non-null    int64  
 5   efficiency                    497 non-null    int64  
 6   num_use_species               497 non-null    int64  
 7   width                         309 non-null    float64
 8   length                        309 non-null    float64
 9   hiehgt                        44 non-null     float64
 10  openness                      44 non-null     float64
 11  solid_slope                   497 non-null    object 
 12  frequency_roadkill            497 non-null    int64  
 13  dist_

## Type Split

In [86]:
df_overpass = df[df['type'] == 0] # 육교형
df_tunnel = df[df['type'] != 0] # 터널형

# 육교형에서 사용하지 않는 열 삭제
df_overpass.drop(['hiehgt', 'openness'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [87]:
# index 초기화
df_overpass.reset_index(drop = True,inplace= True)
df_tunnel.reset_index(drop = True,inplace= True)

In [88]:
df_overpass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   num                           300 non-null    int64  
 1   latitude                      300 non-null    float64
 2   longitude                     300 non-null    float64
 3   type                          300 non-null    int64  
 4   frequency_use_hikers          300 non-null    int64  
 5   efficiency                    300 non-null    int64  
 6   num_use_species               300 non-null    int64  
 7   width                         264 non-null    float64
 8   length                        264 non-null    float64
 9   solid_slope                   300 non-null    object 
 10  frequency_roadkill            300 non-null    int64  
 11  dist_walkway                  300 non-null    float64
 12  dist_farm                     300 non-null    float64
 13  speed

In [89]:
df_tunnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   num                           197 non-null    int64  
 1   latitude                      197 non-null    float64
 2   longitude                     197 non-null    float64
 3   type                          197 non-null    int64  
 4   frequency_use_hikers          197 non-null    int64  
 5   efficiency                    197 non-null    int64  
 6   num_use_species               197 non-null    int64  
 7   width                         45 non-null     float64
 8   length                        45 non-null     float64
 9   hiehgt                        43 non-null     float64
 10  openness                      43 non-null     float64
 11  solid_slope                   197 non-null    object 
 12  frequency_roadkill            197 non-null    int64  
 13  dist_

## Drop unnecessary columns

In [90]:
drop_col = ['num', 'latitude', 'longitude', 'type', 'num_use_species', 'frequency_roadkill', 'fence_latitude', 'fence_longitude']

df_overpass = df_overpass.drop(drop_col, axis=1)
df_tunnel = df_tunnel.drop(drop_col, axis=1)

## One-Hot Encoding

In [91]:
onehot_col = ['frequency_use_hikers', 'solid_slope', 'topography', 'environmental_score']

df_overpass = pd.get_dummies(df_overpass, columns=onehot_col)
df_tunnel = pd.get_dummies(df_tunnel, columns=onehot_col)

## 이상치

### 이상치 확인

In [92]:
# 이상치 처리해야 할 컬럼 목록
overpass_check = ['num_lane', 'width', 'length', 'dist_walkway', 'dist_farm',
                  'speed_limit', 'num_near_species', 'frequency_appearance_animals', 'dist_river', 
                  'dist_hikingtrails', 'fence_length', 'fence_height', 'traffic_volumne', 'dist_buildings']

tunnel_check = ['num_lane', 'width', 'length', 'hiehgt', 'openness', 'dist_walkway', 'dist_farm',
                'speed_limit', 'num_near_species', 'frequency_appearance_animals', 'dist_river', 
                'dist_hikingtrails', 'fence_length', 'fence_height', 'traffic_volumne', 'dist_buildings']
print(len(overpass_check))
print(len(tunnel_check))

14
16


In [93]:
# # 육교형 생태통로 Boxplot
# plt.figure(figsize=(16, 10))

# for i, col in enumerate(overpass_check):
#   plt.subplot(3, 6, i+1)
#   df_overpass.boxplot(col)

# plt.tight_layout()
# plt.show()

In [94]:
# # 터널형 생태통로 Boxplot
# plt.figure(figsize=(16, 10))

# for i, col in enumerate(tunnel_check):
#   plt.subplot(4, 5, i+1)
#   df_tunnel.boxplot(col)

# plt.tight_layout()
# plt.show()

### 이상치 처리

tukey fence 방식으로 처리

In [95]:
# 높은 이상치는 Boxplot의 상한값으로, 낮은 이상치는 Boxplot의 하한값으로 대체하는 함수
def change_outlier(df, col_list):
  for col_name in col_list:
      Q1 = df[col_name].quantile(q=0.25)
      Q3 = df[col_name].quantile(q=0.75)
      IQR = Q3-Q1

      print('[' + col_name + ']', end=' ')
      high_idx = df[df[col_name] > (Q3+1.5*IQR)].index
      df.loc[high_idx, col_name] = round(Q3+1.5*IQR, 1)
      print(str(len(high_idx)) +'개가 상한값으로 변경됨', end=' / ')

      low_idx = df[df[col_name] < (Q1-1.5*IQR)].index
      df.loc[low_idx, col_name] = round(Q1-1.5*IQR, 1)
      print(str(len(low_idx)) +'개가 하한값으로 변경됨')

  return df

In [96]:
df_overpass = change_outlier(df_overpass, overpass_check)
df_tunnel = change_outlier(df_tunnel, tunnel_check)

[num_lane] 8개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[width] 21개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[length] 9개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_walkway] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_farm] 26개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[speed_limit] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[num_near_species] 3개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[frequency_appearance_animals] 19개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_river] 15개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_hikingtrails] 16개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[fence_length] 59개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[fence_height] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[traffic_volumne] 28개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_buildings] 23개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[num_lane] 5개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[width] 2개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[length] 2개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[hiehgt] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[openness] 1개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_walkway] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[dist_farm] 27개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[speed_limit] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[num_near_species] 5개가 상한

In [97]:
# 이상치 처리된 MainData DF → DB로 업로드
# df_overpass.to_sql(name='MainData_outlier_overpass', con=db_connection, if_exists='replace', index=False)
# df_tunnel.to_sql(name='MainData_outlier_tunnel', con=db_connection, if_exists='replace', index=False)

## 결측치

In [98]:
# 유도울타리 높이, 연장, 주변동물 출현빈도, 주변 동물종 개수 => 존재하지 않는 것이므로 0으로 처리
df_overpass[['fence_length', 'fence_height', 'frequency_appearance_animals', 'num_near_species']].fillna(0, inplace=True)
df_tunnel[['fence_length', 'fence_height', 'frequency_appearance_animals', 'num_near_species']].fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


### 규격(폭, 연장, 높이, 개방도)
Case 1. 평균으로 대체

In [99]:
# 규격 데이터의 결측치를 평균으로 대체하는 함수
def MeanImputation(df, col_list):
  df_copy = df.copy()
  for col_name in col_list:
    df_copy[col_name].fillna(np.round(df_copy[col_name].mean(), 1), inplace=True)
  return df_copy

Case 2. KNN으로 대체

In [100]:
# KNN 알고리즘으로 결측값을 대체 함수
def KNNImputation(df, n_neighbors):
  df_copy = df.copy()
  col = df_copy.columns
  imputer = KNNImputer(n_neighbors=n_neighbors)
  df_imputed = imputer.fit_transform(df_copy)
  df_copy = pd.DataFrame(df_imputed)
  return df_copy, col

In [101]:
# [Case 1] 평균으로 대체
df_overpass_mean = MeanImputation(df_overpass, ['width', 'length', 'traffic_volumne'])
df_tunnel_mean = MeanImputation(df_tunnel, ['width', 'length', 'hiehgt', 'openness', 'traffic_volumne'])

In [102]:
# [Case 2] KNN으로 대체

df_over, col_over = KNNImputation(df_overpass, 3)
df_over.columns = col_over
globals()['df_overpass_knn3'] = df_over

df_tun, col_tun = KNNImputation(df_tunnel, 3)
df_tun.columns = col_tun
globals()['df_tunnel_knn3'] = df_tun

## 생태통로 효율성 '판단불가' 대체

Case 1. '판단불가' 값이 있는 행 삭제

In [103]:
# 생태통로 효율성 데이터가 '판단불가' 값인 행을 삭제하는 함수
def drop_value(df) :
    df_copy = df.copy() # 파라미터로 넣은 데이터프레임 원본이 바뀌지 않도록 복사
    idx = list(df_copy[df_copy['efficiency'] == 3].index) # 효율성 3 : 판단불가
    df_copy.drop(idx, inplace=True)
    return df_copy

In [104]:
# 생태통로 효율성 데이터가 '판단불가' 값이 아닌 행을 삭제하는 함수
def drop_value_not3(df) :
    df_copy = df.copy() # 파라미터로 넣은 데이터프레임 원본이 바뀌지 않도록 복사
    idx = list(df_copy[df_copy['efficiency'] != 3].index) # 효율성 3 : 판단불가
    df_copy.drop(idx, inplace=True)
    return df_copy

Case 2. '판단불가' 값을 중앙값('보통')으로 대체

In [105]:
# 생태통로 효율성 데이터가 '판단불가' 값이면 중앙값('보통')으로 변경하는 함수
def change_median(df):
  df_copy=df.copy()
  df_copy['efficiency']=[1 if x==3 else x for x in df_copy['efficiency']]
  return df_copy

In [106]:
# 결측치 : 평균 / 효율성 : '판단불가' 값 삭제 & '판단불가' 값 제외 삭제

overpass_mean_drop = drop_value(df_overpass_mean)
overpass_mean_drop_not3 = drop_value_not3(df_overpass_mean)

tunnel_mean_drop = drop_value(df_tunnel_mean)
tunnel_mean_drop_not3 = drop_value_not3(df_tunnel_mean)

In [107]:
# 결측치 : KNN / 효율성 : '판단불가' 값 삭제 & '판단불가' 값 제외 삭제

overpass_knn3_drop = drop_value(df_overpass_knn3)
overpass_knn3_drop_not3 = drop_value_not3(df_overpass_knn3)

tunnel_knn3_drop = drop_value(df_tunnel_knn3)
tunnel_knn3_drop_not3 = drop_value_not3(df_tunnel_knn3)

In [108]:
# 결측치 : 평균 / 효율성 : 중앙값('보통')

overpass_mean_median = change_median(df_overpass_mean)
tunnel_mean_median = change_median(df_tunnel_mean)

In [109]:
# 결측치 : 평균 / 효율성 : 중앙값('보통')

overpass_knn3_median = change_median(df_overpass_knn3)
tunnel_knn3_median = change_median(df_tunnel_knn3)

## Final Data

In [110]:
df_list = [overpass_mean_drop, overpass_mean_drop_not3, tunnel_mean_drop, tunnel_mean_drop_not3, overpass_mean_median, tunnel_mean_median, 
           overpass_knn3_drop, overpass_knn3_drop_not3, tunnel_knn3_drop, tunnel_knn3_drop_not3, overpass_knn3_median, tunnel_knn3_median]        
print(len(df_list))
print([df.shape for df in df_list])

12
[(183, 39), (117, 39), (122, 41), (75, 41), (300, 39), (197, 41), (183, 39), (117, 39), (122, 41), (75, 41), (300, 39), (197, 41)]


In [111]:
# DF to CSV
path = '/content/drive/MyDrive/데이터청년캠퍼스/Team2_Data File/maindata/'

overpass_mean_drop.to_csv(path + 'overpass_mean_drop.csv', index=False, encoding='cp949')
tunnel_mean_drop.to_csv(path + 'tunnel_mean_drop.csv', index=False, encoding='cp949')
overpass_mean_drop_not3.to_csv(path + 'overpass_mean_drop_not3.csv', index=False, encoding='cp949')
tunnel_mean_drop_not3.to_csv(path + 'tunnel_mean_drop_not3.csv', index=False, encoding='cp949')

overpass_mean_median.to_csv(path + 'overpass_mean_median.csv', index=False, encoding='cp949')
tunnel_mean_median.to_csv(path + 'tunnel_mean_median.csv', index=False, encoding='cp949')

overpass_knn3_drop.to_csv(path + 'overpass_knn3_drop.csv', index=False, encoding='cp949')
tunnel_knn3_drop.to_csv(path + 'tunnel_knn3_drop.csv', index=False, encoding='cp949')
overpass_knn3_drop_not3.to_csv(path + 'overpass_knn3_drop_not3.csv', index=False, encoding='cp949')
tunnel_knn3_drop_not3.to_csv(path + 'tunnel_knn3_drop_not3.csv', index=False, encoding='cp949')

overpass_knn3_median.to_csv(path + 'overpass_knn3_median.csv', index=False, encoding='cp949')
tunnel_knn3_median.to_csv(path + 'tunnel_knn3_median.csv', index=False, encoding='cp949')